In [245]:
''' day 7 '''
import numpy as np 
import pandas as pd

with open('data/day7.txt', 'r') as day7:
    data = [line.strip().split(' ') for line in day7]
data = [[[*hand], bid] for hand, bid in data]
data = [[i] + sublist for i, sublist in enumerate(data)]

print(data)

[[0, ['3', 'J', '4', 'K', 'T'], '513'], [1, ['Q', '3', 'K', '4', '2'], '147'], [2, ['2', '9', 'J', 'Q', 'K'], '187'], [3, ['A', 'A', 'K', 'A', '9'], '821'], [4, ['5', '5', '5', 'J', '5'], '571'], [5, ['6', '4', 'J', 'Q', '2'], '388'], [6, ['A', '2', '9', 'J', '2'], '877'], [7, ['3', '3', '3', '9', '9'], '59'], [8, ['A', 'J', '7', 'A', '8'], '312'], [9, ['3', 'A', 'A', '8', '3'], '474'], [10, ['9', 'J', '9', '7', '9'], '218'], [11, ['7', '8', '4', '7', '8'], '432'], [12, ['9', '6', '6', '4', '4'], '502'], [13, ['J', 'K', 'K', '8', 'T'], '647'], [14, ['3', '3', '9', '2', '4'], '127'], [15, ['5', '6', 'J', '4', '2'], '864'], [16, ['2', '8', 'J', '6', '6'], '208'], [17, ['4', '4', '4', '9', '4'], '469'], [18, ['9', '3', '9', '3', '9'], '167'], [19, ['6', '2', '2', '2', '4'], '191'], [20, ['4', 'T', '3', 'T', 'T'], '447'], [21, ['6', '9', 'J', '9', '9'], '217'], [22, ['7', 'A', 'T', 'J', '7'], '99'], [23, ['8', '3', '2', '5', 'T'], '792'], [24, ['T', '3', '4', '4', '4'], '545'], [25, ['9', 

In [246]:
''' helper functions '''

### turns a 5 card hand into a list of numbers that correspoind 
### to how many of each card is in the hand

def encode_hand(hand):
    '''turns a 5 card hand into a list of numbers that correspoind 
        to how many of each card is in the hand 
    '''
    
    a, b, c, d, e = [0]*5
    uniq = list(u:=np.unique(hand)) + ['DV']*(5-len(u))
    
    for i in hand:
        a += (i == uniq[0])
        b += (i == uniq[1])
        c += (i == uniq[2])
        d += (i == uniq[3])
        e += (i == uniq[4])
    return sorted([a, b, c, d, e], reverse = True)



def bin_hands(hand):
    '''
        based on the encoding, this function assigns the cards to their proper group:
        7 = 5 of a kind
        6 = 4 of a kind
        5 = full house
        4 = 3 of a kind
        3 = two pairs
        2 = two of a kind
        1 = all different 
    '''
    
    encoding = encode_hand(hand[1])
    
    if encoding[0] == 5:
        return 7
    elif encoding[0] == 4:
        return 6
    elif encoding[0] == 3:
        if encoding[1] == 2:
            return 5
        else:
            return 4
    elif encoding[0] == 2:
        if encoding[1] == 2:
            return 3
        else:
            return 2
    else:
        return 1

In [247]:
### create np.array of the dataset
data_array = np.array([[data[i][0], data[i][2]] + data[i][1] for i in range(len(data))])

### translate to pandas DataFrame
df = pd.DataFrame(data_array, columns = ['ID', 'bid','card_1', 'card_2', 'card_3', 'card_4', 'card_5']) #############
df.ID = df.ID.astype('int')
df.bid = df.bid.astype('int')

### define dictionary to map card names to ints
card_map = {'2':2,
         '3':3,
         '4':4,
         '5':5,
         '6':6,
         '7':7,
         '8':8,
         '9':9,
         'T':10,
         'J':11,
         'Q':12,
         'K':13,
         'A':14}

### apply the card_map
for col in df.columns[1:]:
    df[col] = df[col].replace(card_map)

### use the bin_hands function to label each hand with the corresponding group
df['group'] = df.apply(lambda x: bin_hands(data[x.ID]), axis = 1)

#sort the df according to group, then card values
df = df.sort_values(['group', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5'], ascending = False)#.iloc[::-1].reset_index()

# reset aindex accordingly
df['ID'] = list(range(1, len(data) + 1))[::-1]
df.head()

,ID,bid,card_1,card_2,card_3,card_4,card_5,group
647,1000,287,11,11,11,11,11,7
327,999,130,14,14,14,14,11,6
597,998,819,14,14,14,12,14,6
994,997,997,14,14,9,14,14,6
153,996,690,14,13,14,14,14,6


In [248]:
answer = sum([df.ID.iloc[i] * df.bid.iloc[i] for i in range(len(data))])
print(answer)

250453939


In [250]:
'''part 2'''

''' helper functions '''

### turns a 5 card hand into a list of numbers that correspoind 
### to how many of each card is in the hand

def encode_hand(hand):
    '''turns a 5 card hand into a list of numbers that correspoind 
        to how many of each card is in the hand 
    '''
    
    a, b, c, d, e = [0]*5
    
    unique_elements, first_indices = np.unique(hand, return_index=True)
    # Sort the unique elements by their first occurrence
    uniq = list(u:=unique_elements[np.argsort(first_indices)]) + ['DV']*(5-len(u))

    
    for i in hand:
        a += (i == uniq[0])
        b += (i == uniq[1])
        c += (i == uniq[2])
        d += (i == uniq[3])
        e += (i == uniq[4])
    
    encoding = [a, b, c, d, e]
    
    if ('J' in uniq) and (len(uniq) > 1):
        idx = uniq.index('J')
        temp = encoding[idx]
        encoding[idx] = 0
        encoding = sorted(encoding, reverse = True)
        encoding[0] += temp
    else:
        encoding = sorted(encoding, reverse = True)
        
    return encoding



def bin_hands(hand):
    '''
        based on the encoding, this function assigns the cards to their proper group:
        7 = 5 of a kind
        6 = 4 of a kind
        5 = full house
        4 = 3 of a kind
        3 = two pairs
        2 = two of a kind
        1 = all different 
    '''
    
    encoding = encode_hand(hand[1])
    
    if encoding[0] == 5:
        return 7
    elif encoding[0] == 4:
        return 6
    elif encoding[0] == 3:
        if encoding[1] == 2:
            return 5
        else:
            return 4
    elif encoding[0] == 2:
        if encoding[1] == 2:
            return 3
        else:
            return 2
    else:
        return 1


In [252]:
### create np.array of the dataset
data_array = np.array([[data[i][0], data[i][2]] + data[i][1] for i in range(len(data))])

### translate to pandas DataFrame
df = pd.DataFrame(data_array, columns = ['ID', 'bid','card_1', 'card_2', 'card_3', 'card_4', 'card_5']) #############
df.ID = df.ID.astype('int')
df.bid = df.bid.astype('int')

### define dictionary to map card names to ints
card_map = {'2':2,
         '3':3,
         '4':4,
         '5':5,
         '6':6,
         '7':7,
         '8':8,
         '9':9,
         'T':10,
         'J':1,
         'Q':12,
         'K':13,
         'A':14}

### apply the card_map
for col in df.columns[1:]:
    df[col] = df[col].replace(card_map)

### use the bin_hands function to label each hand with the corresponding group
df['group'] = df.apply(lambda x: bin_hands(data[x.ID]), axis = 1)

#sort the df according to group, then card values
df = df.sort_values(['group', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5'], ascending = False)#.iloc[::-1].reset_index()

# reset aindex accordingly
df['ID'] = list(range(1, len(data) + 1))[::-1]


In [253]:
answer = sum([df.ID.iloc[i] * df.bid.iloc[i] for i in range(len(data))])
print(answer)

248652697
